<h3> Function to calculate Balassa index of patent radicality </h3>

<p> Matt Wilder, University of Toronto <br>
Please address questions and comments to <a href="mailto:matt.wilder@utoronto.ca">matt.wilder@utoronto.ca</a>. </p>

<p>Updated 9 May 2024</p>

This function calculates the Balassa Index of patent radicality, as used as a measure of revealed comparative advantage (RCA) in Meelan et al (2017) 'Disentangling patterns of economic, technological and innovative specialization of Western economies: An assessment of the Varieties-of-Capitalism theory on comparative institutional advantages' <i>Research Policy, 46</i>(3): 667-77.

The Radicality Index (RI) is defined as follows:<br><br>
$$
\large RI_{ijt} = \frac{\frac{Y_{ijt}^*}{Y_{ijt} + Y_{ijt}^*}}{\frac{\sum_{i=1}^{n} Y_{ijt}^*}{\sum_{i=1}^{n} (Y_{ijt} + Y_{ijt}^*)}}
$$

where:
- $RI_{ijt}$ is the Radicality Index for country $i$ in sector $j$ at time $t$.
- $Y^*_{ijt}$ denotes the number of radical patents for country $i$ in sector $j$ at time $t$.
- $Y_{ijt}$ refers to the number of non-radical patents of country $i$ in sector $j$ at time $t$.
- The denominator is the ratio of the sum of radical patents across all countries to the sum of total patents across all countries in sector $j$ at time $t$. 


This index captures the relative specialization of countries in radical innovation within specific sectors. Technically, the range of the variable is 0 to $\infty$. Conventional interpretation is as follows:
<ul>
<li>RI > 1: the country has a revealed comparative advantage in the sector. </li>
<li>RI < 1: the country has a revealed comparative disadvantage in the sector.</li>
</ul>


Meelan et al (2017) normalize the distrubution between -1 and 1, whereby values below zero signify a comparative disadvantage, while values above zero signify a comparative advantage. 

$$
\normalsize RI' = \frac{RI - 1}{RI + 1}
$$

<ul>
<li>RI' > 0: the country has a revealed comparative advantage in the sector. </li>
<li>RI' < 0: the country has a revealed comparative disadvantage in the sector. </li>
</ul>


In [1]:
import pandas as pd
import numpy as np
import ast
import statsmodels.api as sm
import statsmodels.formula.api as smf

'''load in the sample data (this may take a few minutes if the connection is slow)'''

df = pd.read_csv('https://raw.githubusercontent.com/matt-wilder/patent-research/main/forward_citations_with_cpc_classifications_sample.csv',sep='\t', dtype='str')
df

,patent,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode,name,date,year,forward_citations,generality_hhi,patent_cpcs,country,VoC
0,4240895,{'US': 1.0},{},['US'],NaN,Lees,1991-02-01,1991,11.0,0.88,C25B|C25B|C25B|C25B|C25B|C25B|C25B|C25B|C25B|C...,US,LME
1,4893178,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Matama et al.,1990-01-01,1990,11.0,0.56,G03B|H04N|G03B|H04N|G03B|H04N|G03B|H04N|G03B|H...,JP,CME
2,4896067,{'DE': 1.0},{'DE': 1.0},['DE'],['DE'],Walther,1990-01-01,1990,14.0,0.66,H02K|H02K|H02K|H02K|H02K|H02K|H02K|H02K|H02K|H...,DE,CME
3,4900212,{'JP': 1.0},{'US': 1.0},['JP'],['US'],Mikahara,1990-02-01,1990,18.0,0.76,Y10S|H01L|Y10S|H01L|Y10S|H01L|Y10S|H01L|Y10S|H...,US,LME
4,4903398,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Sakamoto et al.,1990-02-01,1990,11.0,0.72,G01M|Y10T|G01M|Y10T|G01M|Y10T|G01M|Y10T|G01M|Y...,JP,CME
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,D423357,{},{},NaN,NaN,Chrisco et al.,2000-04-01,2000,4.0,0.0,NaN,US,LME
496,D427997,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Inoue,2000-07-01,2000,8.0,0.0,NaN,JP,CME
497,D430278,{},{},NaN,NaN,Krauss et al.,2000-08-01,2000,14.0,0.0,NaN,US,LME
498,D433628,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Tamura,2000-11-01,2000,5.0,0.69,NaN,JP,CME


In [3]:
''' define the tech class '''

'''by the section codes (8 unique in this dataset)'''
df['tech_class'] = df['patent_cpcs'].str.split('|').str[0].str[0]

# A - Human Necessities (e.g., Health, Food)
# B - Performing Operations; Transporting (e.g., Manufacturing, Vehicles)
# C - Chemistry; Metallurgy (e.g., Chemicals, Alloys)
# D - Textiles; Paper (e.g., Fabrics, Papermaking)
# E - Fixed Constructions (e.g., Buildings, Infrastructure)
# F - Mechanical Engineering; Lighting; Heating; Weapons; Blasting (e.g., Machines, HVAC)
# G - Physics (e.g., Optics, Instruments)
# H - Electricity (e.g., Circuits, Communications)

'''or by the classification codes (82 unqiue classes in this dataset)'''
# df['tech_class'] = df['patent_cpcs'].str.split('|').str[0].str[:3]

'''or using fractional counting'''
def process_patent(row):
    if pd.notna(row['patent_cpcs']):  # check if 'patent_cpcs' is not NaN
        cpcs = row['patent_cpcs'].split('|')

        # count occurrences of each CPC
        cpc_counts = {}
        for cpc in set(cpcs):
            cpc_counts[cpc] = cpcs.count(cpc)

        # calculate fractional contributions for each CPC, rounding to two decimal places
        total_cpcs = len(cpcs)
        cpc_contributions = {cpc: round(count / total_cpcs, 2) for cpc, count in cpc_counts.items()}

        return cpc_contributions
    else:
        return {}
df['frac_class'] = df.apply(process_patent, axis=1)

df

,patent,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode,name,date,year,forward_citations,generality_hhi,patent_cpcs,country,VoC,tech_class,frac_class
0,4240895,{'US': 1.0},{},['US'],NaN,Lees,1991-02-01,1991,11.0,0.88,C25B|C25B|C25B|C25B|C25B|C25B|C25B|C25B|C25B|C...,US,LME,C,{'C25B': 1.0}
1,4893178,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Matama et al.,1990-01-01,1990,11.0,0.56,G03B|H04N|G03B|H04N|G03B|H04N|G03B|H04N|G03B|H...,JP,CME,G,"{'H04N': 0.5, 'G03B': 0.5}"
2,4896067,{'DE': 1.0},{'DE': 1.0},['DE'],['DE'],Walther,1990-01-01,1990,14.0,0.66,H02K|H02K|H02K|H02K|H02K|H02K|H02K|H02K|H02K|H...,DE,CME,H,{'H02K': 1.0}
3,4900212,{'JP': 1.0},{'US': 1.0},['JP'],['US'],Mikahara,1990-02-01,1990,18.0,0.76,Y10S|H01L|Y10S|H01L|Y10S|H01L|Y10S|H01L|Y10S|H...,US,LME,Y,"{'Y10S': 0.5, 'H01L': 0.5}"
4,4903398,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Sakamoto et al.,1990-02-01,1990,11.0,0.72,G01M|Y10T|G01M|Y10T|G01M|Y10T|G01M|Y10T|G01M|Y...,JP,CME,G,"{'Y10T': 0.5, 'G01M': 0.5}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,D423357,{},{},NaN,NaN,Chrisco et al.,2000-04-01,2000,4.0,0.0,NaN,US,LME,NaN,{}
496,D427997,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Inoue,2000-07-01,2000,8.0,0.0,NaN,JP,CME,NaN,{}
497,D430278,{},{},NaN,NaN,Krauss et al.,2000-08-01,2000,14.0,0.0,NaN,US,LME,NaN,{}
498,D433628,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Tamura,2000-11-01,2000,5.0,0.69,NaN,JP,CME,NaN,{}


In [4]:
'''calculate radicality measure'''

# function to calculate deciles for radicality measures by tech class and time period (window based on the number of patents required to populate a decile distribution, per "window_" variables)
def calculate_deciles_for_patents(df, indicators, year_column='year', tech_class_column='tech_class'): #tech class based on first listed tech class, not fractional count
    # ensure df has a unique index
    df.reset_index(inplace=True, drop=True)
    
    # store the decile and time frame ('window') information for each indicator
    deciles_info = pd.DataFrame(index=df.index)

    # iterate over each indicator, calculating deciles and windows separately
    for indicator in indicators:
        # initialize a list to store windows for each indicator
        deciles_info[f'window_{indicator}'] = pd.NA
        
        # filter out NaNs and values < 0
        indicator_df = df.dropna(subset=[indicator])
        indicator_df = indicator_df[indicator_df[indicator] > 0]

        # ensure year is treated as integer
        indicator_df[year_column] = indicator_df[year_column].astype(int)

        # group by tech class and sort unique years within each tech class for each indicator
        for tech_class, group in indicator_df.groupby(tech_class_column):
            years = sorted(group[year_column].unique())
            grouped_years = []

            # process each year within the tech class
            for year in years:
                if year not in grouped_years:
                    current_group = [year]
                    while True:
                        temp_df = group[group[year_column].isin(current_group)].copy()
                        if len(temp_df) >= 10 or (min(current_group) == min(years) and max(current_group) == max(years)):
                            window = f"{min(current_group)} - {max(current_group)}"
                            grouped_years.extend(current_group)
                            
                            # calculate deciles for the current group
                            decile_label = f'{indicator}_decile'
                            temp_df[decile_label] = pd.qcut(temp_df[indicator].rank(method='first'), 10, labels=range(1, 11), duplicates='drop')
                            temp_df[f'window_{indicator}'] = window  # Create the column to store the window information
                            
                            # update the deciles_info df with decile and window info
                            deciles_info.loc[temp_df.index, decile_label] = temp_df[decile_label]
                            deciles_info.loc[temp_df.index, f'window_{indicator}'] = window
                            
                            break
                        else:
                            if min(current_group) > min(years):
                                current_group.insert(0, min(current_group) - 1)
                            if max(current_group) < max(years):
                                current_group.append(max(current_group) + 1)
                            if min(current_group) <= min(years) and max(current_group) >= max(years):
                                break

    # concatenate the original df with the deciles_info df
    result_df = pd.concat([df, deciles_info], axis=1)

    return result_df

indicators = ['forward_citations', 'generality_hhi']  # these indicators need to be converted to numeric for the calculation to work
df[['forward_citations', 'generality_hhi']] = df[['forward_citations', 'generality_hhi']].apply(pd.to_numeric, errors='coerce')

df = calculate_deciles_for_patents(df, indicators) # note the OR statement and decile specification; set these as desired
df['is_radical'] = ((df['forward_citations_decile'] == 10) | (df['generality_hhi_decile'] == 10)).astype(int)

df

,patent,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode,name,date,year,forward_citations,generality_hhi,patent_cpcs,country,VoC,tech_class,frac_class,window_forward_citations,forward_citations_decile,window_generality_hhi,generality_hhi_decile,is_radical
0,4240895,{'US': 1.0},{},['US'],NaN,Lees,1991-02-01,1991,11.0,0.88,C25B|C25B|C25B|C25B|C25B|C25B|C25B|C25B|C25B|C...,US,LME,C,{'C25B': 1.0},1990 - 1991,7,1990 - 1991,9,0
1,4893178,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Matama et al.,1990-01-01,1990,11.0,0.56,G03B|H04N|G03B|H04N|G03B|H04N|G03B|H04N|G03B|H...,JP,CME,G,"{'H04N': 0.5, 'G03B': 0.5}",1990 - 1991,4,1990 - 1991,2,0
2,4896067,{'DE': 1.0},{'DE': 1.0},['DE'],['DE'],Walther,1990-01-01,1990,14.0,0.66,H02K|H02K|H02K|H02K|H02K|H02K|H02K|H02K|H02K|H...,DE,CME,H,{'H02K': 1.0},1990 - 1991,6,1990 - 1991,5,0
3,4900212,{'JP': 1.0},{'US': 1.0},['JP'],['US'],Mikahara,1990-02-01,1990,18.0,0.76,Y10S|H01L|Y10S|H01L|Y10S|H01L|Y10S|H01L|Y10S|H...,US,LME,Y,"{'Y10S': 0.5, 'H01L': 0.5}",<NA>,NaN,<NA>,NaN,0
4,4903398,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Sakamoto et al.,1990-02-01,1990,11.0,0.72,G01M|Y10T|G01M|Y10T|G01M|Y10T|G01M|Y10T|G01M|Y...,JP,CME,G,"{'Y10T': 0.5, 'G01M': 0.5}",1990 - 1991,4,1990 - 1991,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,D423357,{},{},NaN,NaN,Chrisco et al.,2000-04-01,2000,4.0,0.00,NaN,US,LME,NaN,{},NaN,NaN,NaN,NaN,0
496,D427997,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Inoue,2000-07-01,2000,8.0,0.00,NaN,JP,CME,NaN,{},NaN,NaN,NaN,NaN,0
497,D430278,{},{},NaN,NaN,Krauss et al.,2000-08-01,2000,14.0,0.00,NaN,US,LME,NaN,{},NaN,NaN,NaN,NaN,0
498,D433628,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Tamura,2000-11-01,2000,5.0,0.69,NaN,JP,CME,NaN,{},NaN,NaN,NaN,NaN,0


<h3>Crude Balassa index of innovation advantage</h3>
<br>
$$
\large RI_{ijt} = \frac{\frac{Y_{ijt}^*}{Y_{ijt} + Y_{ijt}^*}}{\frac{\sum_{i=1}^{n} Y_{ijt}^*}{\sum_{i=1}^{n} (Y_{ijt} + Y_{ijt}^*)}}
$$

$Y^*_{ijt}$ = number of 10<sup>th</sup> decile patents in country $i$ in tech class $j$ in year $t$.<br>
$Y_{ijt}$ = number of all other patents in country $i$ in tech class $j$ in year $t$. <br>
The denominator is the ratio of the sum of radical patents across all countries to the sum of total patents across all countries in sector $j$ at time $t$.


In [6]:
'''calculate a (crude) Radicality (Balassa) Index for each country i in tech class j at time t'''

# Step 1: aggregate data
aggregated_data = df.groupby(['country', 'tech_class', 'year', 'is_radical']).size().unstack(fill_value=0)
aggregated_data.reset_index(inplace=True)
aggregated_data.rename(columns={0: 'non_radical_patents', 1: 'radical_patents'}, inplace=True)

# Step 2: calculate the RI numerator for each country, tech class, and year
aggregated_data['numerator'] = aggregated_data['radical_patents'] / (aggregated_data['non_radical_patents'] + aggregated_data['radical_patents'])

# Step 3: calculate the RI denominator
total_radical_patents = aggregated_data.groupby(['tech_class', 'year'])['radical_patents'].sum() # sum of radical patents across all countries
total_patents = aggregated_data.groupby(['tech_class', 'year']).apply(lambda x: (x['non_radical_patents'] + x['radical_patents']).sum()) # sum of total patents across all countries
ri_denominator = total_radical_patents / total_patents # ratio of the sums for the denominator

# map the calculated denominator back to the aggregated data
aggregated_data['denominator'] = aggregated_data.apply(lambda row: ri_denominator.get((row['tech_class'], row['year'])), axis=1)

# Ensure the denominator is never zero or NaN (replace 0 with a very small number, e.g., 1e-10, to avoid division by zero)
aggregated_data['denominator'] = aggregated_data['denominator'].replace(0, 1e-10)

# Step 4: compute RI
aggregated_data['RI'] = aggregated_data['numerator'] / aggregated_data['denominator']

# Step 5: normalize RI using the transformation formula
aggregated_data['RI_normalized'] = (aggregated_data['RI'] - 1) / (aggregated_data['RI'] + 1)

# display the data frame of Rijt values (RI for each country, tech class, and year)
crude_balassa_df = aggregated_data
crude_balassa_df

# Optional: display only cases of comparative advantage
# crude_balassa_df = crude_balassa_df[crude_balassa_df['RI_normalized'] > 0]

is_radical,country,tech_class,year,non_radical_patents,radical_patents,numerator,denominator,RI,RI_normalized
0,AT,D,1992,1,0,0.000,1.000000e-10,0.000000,-1.000000
1,AU,F,1993,1,0,0.000,2.000000e-01,0.000000,-1.000000
2,BE,G,1991,0,1,1.000,3.333333e-01,3.000000,0.500000
3,CA,A,1992,0,1,1.000,2.500000e-01,4.000000,0.600000
4,CA,A,1993,1,0,0.000,1.000000e-10,0.000000,-1.000000
...,...,...,...,...,...,...,...,...,...
183,US,H,1996,3,2,0.400,2.500000e-01,1.600000,0.230769
184,US,H,1997,2,0,0.000,1.000000e-10,0.000000,-1.000000
185,US,H,1999,3,2,0.400,2.142857e-01,1.866667,0.302326
186,US,H,2000,5,3,0.375,2.083333e-01,1.800000,0.285714


<h3>Using Fractional Counts to Calculate a Balassa Index of Innovation Advantage</h3>

Fractional counts accommodate patents that may be associated with multiple countries and/or technological classes. The approach proportionally distributes each patent's contribution across its associated categories.

Define:
- $f^*_{ijt}$ as the sum of the products of country and CPC class contributions for the radical patents ($R$) for country $i$ in technology class $j$ at time $t$.
- $f_{ijt}$ similarly for non-radical patents ($N$).

The fractional contributions for radical and non-radical patents are calculated as:

$$
f^*_{ijt} = \sum_{p \in R} \left( c_{ip} \times d_{jp} \right)
$$
$$
f_{ijt} = \sum_{p \in N} \left( c_{ip} \times d_{jp} \right)
$$

where $c_{ip}$ denotes the fractional contribution of patent $p$ to country $i$, and $d_{jp}$ denotes the fractional contribution of patent $p$ to CPC class $j$.

The Radicality Index (RI) is then calculated as: 
<br>
$$
\large RI_{ijt} = \frac{\frac{f_{ijt}^*}{f_{ijt} + f_{ijt}^*}}{\frac{\sum_{i=1}^{n} f_{ijt}^*}{\sum_{i=1}^{n} (f_{ijt} + f_{ijt}^*)}}
$$


Normalizing the measure: 
$$
RI' = \frac{RI_{ijt} - 1}{RI_{ijt} + 1}
$$


In [22]:
'''calculate the Radicality Index using fractional counts for country i and tech class j'''

def expand_dict_rows(df, col_name, key_name, value_name):
    """normalize dictionary columns and expand them into rows with specified key and value names."""
    rows = []
    for index, row in df.iterrows():
        dict_data = row[col_name]
        if isinstance(dict_data, str):
            dict_data = ast.literal_eval(dict_data)  # evaluate string literal to dictionary
        if isinstance(dict_data, dict):
            for key, value in dict_data.items():
                new_row = row.to_dict()
                new_row[key_name] = key
                new_row[value_name] = value
                rows.append(new_row)
    return pd.DataFrame(rows)

def calculate_fractional_index(df):
    # expand dictionary data to rows
    country_frac = expand_dict_rows(df[['patent', 'is_radical', 'year', 'inventors_country_frac']], 'inventors_country_frac', 'country', 'country_contribution')
    class_frac = expand_dict_rows(df[['patent', 'is_radical', 'year', 'frac_class']], 'frac_class', 'tech_class', 'class_contribution')

    # merge on keys ensuring correct alignment
    merged_df = pd.merge(country_frac, class_frac, on=['patent', 'is_radical', 'year'])
    
    # calculate weighted contributions
    merged_df['weighted_contribution'] = merged_df['country_contribution'] * merged_df['class_contribution']
    
    # group by keys to aggregate contributions
    grouped = merged_df.groupby(['country', 'tech_class', 'year', 'is_radical'])
    sum_data = grouped['weighted_contribution'].sum().unstack(fill_value=0).reset_index()

    # rename columns for clarity
    sum_data.rename(columns={0: 'non_radical_contributions', 1: 'radical_contributions'}, inplace=True)

    # calculate total radical and total patents for normalization
    total_radical = sum_data.groupby(['tech_class', 'year'])['radical_contributions'].sum()
    total_all = sum_data.groupby(['tech_class', 'year'])['non_radical_contributions'].sum() + total_radical

    # calculate RI for each row, handle zero in denominator
    def calculate_ri(row):
        contributions_sum = row['non_radical_contributions'] + row['radical_contributions']
        total_ratio = total_radical.get((row['tech_class'], row['year']), 0) / total_all.get((row['tech_class'], row['year']), 1)
        if total_ratio == 0:
            # Return zero or another appropriate default value indicating no specialization
            return 0  
        return (row['radical_contributions'] / contributions_sum) / total_ratio

    sum_data['RI'] = sum_data.apply(calculate_ri, axis=1)

    # normalize the RI, handling cases where RI is zero or no normalization needed
    sum_data['RI_normalized'] = sum_data['RI'].apply(lambda x: (x - 1) / (x + 1) if x > 0 else 0)

    return sum_data

fractional_balassa_df = calculate_fractional_index(df)

fractional_balassa_df

is_radical,country,tech_class,year,non_radical_contributions,radical_contributions,RI,RI_normalized
0,AT,B01D,1992,0.20,0.00,0.000000,0.000000
1,AT,C02F,1992,0.20,0.00,0.000000,0.000000
2,AT,C07D,1992,0.20,0.00,0.000000,0.000000
3,AT,D01F,1992,0.20,0.00,0.000000,0.000000
4,AT,Y02P,1992,0.20,0.00,0.000000,0.000000
...,...,...,...,...,...,...,...
729,US,Y10T,1995,0.50,0.00,0.000000,0.000000
730,US,Y10T,1996,1.16,0.83,1.125628,0.059102
731,US,Y10T,1997,1.82,0.50,1.573276,0.222781
732,US,Y10T,1999,1.83,0.50,1.497854,0.199313


<h3>Analysis application</h3>

Meelan et al (2017) analyze the RI over 3 year moving averages 

In [28]:
def add_3_year_rolling_avg(df):
    df.sort_values(by=['country', 'tech_class', 'year'], inplace=True)

    # calculate 3-year moving averages for the normalized RI
    rolling_avg = df.groupby(['country', 'tech_class'])['RI_normalized'].rolling(window=3, min_periods=1).mean()

    # assign the rolling averages to a new column
    df['RI_3yr_avg'] = rolling_avg.values  # Use .values to correctly align data without index issues

    return df

# apply the function to the two dataframes
crude_balassa_df = add_3_year_rolling_avg(crude_balassa_df)
fractional_balassa_df = add_3_year_rolling_avg(fractional_balassa_df)

# display the dataframes to verify the results
print(crude_balassa_df[['country', 'tech_class', 'year', 'RI', 'RI_normalized', 'RI_3yr_avg']])
print(fractional_balassa_df[['country', 'tech_class', 'year', 'RI', 'RI_normalized', 'RI_3yr_avg']])


is_radical country tech_class  year        RI  RI_normalized  RI_3yr_avg
0               AT          D  1992  0.000000      -1.000000   -1.000000
1               AU          F  1993  0.000000      -1.000000   -1.000000
2               BE          G  1991  3.000000       0.500000    0.500000
3               CA          A  1992  4.000000       0.600000    0.600000
4               CA          A  1993  0.000000      -1.000000   -0.200000
..             ...        ...   ...       ...            ...         ...
183             US          H  1996  1.600000       0.230769   -0.226107
184             US          H  1997  0.000000      -1.000000   -0.226107
185             US          H  1999  1.866667       0.302326   -0.155635
186             US          H  2000  1.800000       0.285714   -0.137320
187             US          Y  1990  0.000000      -1.000000   -1.000000

[188 rows x 6 columns]
is_radical country tech_class  year        RI  RI_normalized  RI_3yr_avg
0               AT       B0

In [26]:
'''assign VoC categories'''

inst_dict = {
    'AU': 'LME', 'CA': 'LME', 'GB': 'LME', 'IE': 'LME', 'NZ': 'LME',
    'US': 'US', # set US to its own category per Taylor (2004), Akkermans et al (2009), Meelan et al (2017)
    'AT': 'CME', 'BE': 'CME', 'CH': 'CME', 'DE': 'CME', 'DT': 'CME', 'DK': 'CME',
    'FI': 'CME', 'JP': 'CME', 'JA': 'CME', 'KR': 'CME', 'NL': 'CME', 'NO': 'CME',
    'SE': 'CME', 'PT': 'MME', 'ES': 'MME', 'FR': 'MME', 'IT': 'MME', 'GR': 'MME'}


# create 'VoC' column
crude_balassa_df['VoC'] = crude_balassa_df['country'].map(lambda x: inst_dict.get(x, 'other'))
fractional_balassa_df['VoC'] = fractional_balassa_df['country'].map(lambda x: inst_dict.get(x, 'other'))

In [20]:
'''regression: crude Balassa Index'''

def run_regression_analysis(df):
    # create a copy of the dataframe to preserve the original
    df_copy = df.copy()

    # create dummy variables for 'year' and 'tech_class'
    year_dummies = pd.get_dummies(df_copy['year'], prefix='year')
    tech_class_dummies = pd.get_dummies(df_copy['tech_class'], prefix='tech_class')

    # concatenate these dummies to the main dataframe copy
    df_copy = pd.concat([df_copy, year_dummies, tech_class_dummies], axis=1)

    # convert 'VoC' to a categorical type if it's not already
    df_copy['VoC'] = pd.Categorical(df_copy['VoC'])

    # create a unique group identifier for clustering
    df_copy['cluster_group'] = df_copy['country'] + '_' + df_copy['tech_class']

    # define the formula: Include 'VoC', year dummies, and tech class dummies as independent variables
    independent_vars = ' + '.join(['VoC'] + year_dummies.columns.tolist() + tech_class_dummies.columns.tolist())
    formula = f'RI_3yr_avg ~ {independent_vars}'

    # fit the regression model with clustering
    model = smf.ols(formula, data=df_copy).fit(cov_type='cluster', cov_kwds={'groups': df_copy['cluster_group']})

    return model.summary()

# call the function with data frame of crude Balassa indices
regression_results = run_regression_analysis(crude_balassa_df)
print(regression_results)


                            OLS Regression Results                            
Dep. Variable:             RI_3yr_avg   R-squared:                       0.441
Model:                            OLS   Adj. R-squared:                  0.370
Method:                 Least Squares   F-statistic:                     45.88
Date:                Mon, 13 May 2024   Prob (F-statistic):           4.24e-28
Time:                        20:42:55   Log-Likelihood:                -72.917
No. Observations:                 188   AIC:                             189.8
Df Residuals:                     166   BIC:                             261.0
Df Model:                          21                                         
Covariance Type:              cluster                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -0.8024 

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 23, but rank is 22
  warnings.warn('covariance of constraints does not have full '


In [29]:
'''regression: fractional Balassa Index'''

fractional_balassa_df['tech_dummy'] = fractional_balassa_df['tech_class'].str[0] # Create 'tech_dummy' by extracting the first letter from 'tech_class'

def run_regression_analysis(df):
    # Create a copy of the dataframe to preserve the original
    df_copy = df.copy()

    # Create dummy variables for 'year' and the new 'tech_dummy'
    year_dummies = pd.get_dummies(df_copy['year'], prefix='year')
    tech_dummy_dummies = pd.get_dummies(df_copy['tech_dummy'], prefix='tech')

    # Concatenate these dummies to the main dataframe copy
    df_copy = pd.concat([df_copy, year_dummies, tech_dummy_dummies], axis=1)

    # Convert 'VoC' to a categorical type if it's not already
    df_copy['VoC'] = pd.Categorical(df_copy['VoC'])

    # Create a unique group identifier for clustering
    df_copy['cluster_group'] = df_copy['country'] + '_' + df_copy['tech_dummy']

    # Define the formula: Include 'VoC', year dummies, and tech_dummy dummies as independent variables
    independent_vars = ' + '.join(['VoC'] + year_dummies.columns.tolist() + tech_dummy_dummies.columns.tolist())
    formula = f'RI_3yr_avg ~ {independent_vars}'

    # Fit the regression model with the correct clustering
    model = smf.ols(formula, data=df_copy).fit(cov_type='cluster', cov_kwds={'groups': df_copy['cluster_group']})

    # Print the summary of the model
    return model.summary()

# Call the function with the original dataframe
regression_results = run_regression_analysis(fractional_balassa_df)
print(regression_results)

                            OLS Regression Results                            
Dep. Variable:             RI_3yr_avg   R-squared:                       0.099
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     65.52
Date:                Mon, 13 May 2024   Prob (F-statistic):           3.68e-45
Time:                        20:45:20   Log-Likelihood:                 841.78
No. Observations:                 734   AIC:                            -1640.
Df Residuals:                     712   BIC:                            -1538.
Df Model:                          21                                         
Covariance Type:              cluster                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.0039      0.00

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 23, but rank is 22
  warnings.warn('covariance of constraints does not have full '
